In [1]:
from smiles_spe_tokenizer import SMILES_SPE_Tokenizer
import datasets
import torch
import sys
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import RobertaConfig, RobertaForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer , DataCollatorWithPadding , AutoTokenizer, AutoConfig , AutoModelForCausalLM
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

from datasets import Dataset
from datasets import load_dataset, load_metric
import os
import wandb
import torch
import evaluate
from bertviz_repo.bertviz import head_view
from CollatorDatasetTrainer import CustomDataCollator
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from CollatorDatasetTrainer import CustomDataCollator , ToxicTrainer
import numpy as np
from utils import *
from utils import split_stratified_into_train_val_test
from datasets import load_metric
# from transformers.integrations import EarlyStoppingCallback
# import balanced accuracy score
from sklearn.metrics import balanced_accuracy_score , accuracy_score , precision_score , recall_score , f1_score , roc_auc_score
from bert_loves_chemistry.chemberta.utils.molnet_dataloader import load_molnet_dataset, write_molnet_dataset_for_chemprop
# torch.cuda.is_available()
# os.environ["WANDB_NOTEBOOK_NAME"] = 'Molecular_language_models.ipynb'

2023-06-29 01:27:19.109677: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/isiddig/anaconda3/envs/toxicity/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# prepare the tokenizers
Spe_tokenizer = SMILES_SPE_Tokenizer(vocab_file='vocabs/spe_vocab/vocab_speSMILESDB_1000.txt', spe_file= 'vocabs/spe_vocab/cleaned_smilesDB_1000.smi', tokenizer_type='SPE')
# use white space tokenizer
MacFrag_tokenizer = SMILES_SPE_Tokenizer(vocab_file='vocabs/macFrag_vocab/smilesDB_vocab.smi', tokenizer_type='MacFrag')

Bpe_model_pt = 'models/tokenizers/bpe_DBsmi_1000.bin'
Bpe_vocab_pt = 'vocabs/bpe_vocab/bpe_DBsmi_vocab_1000.txt'
Bpe_tokenizer = SMILES_SPE_Tokenizer(vocab_file= Bpe_vocab_pt, 
                                     tokenizer_path= Bpe_model_pt,
                                     tokenizer_type='BPE')

morf_model_pt='models/tokenizers/morf_smilesDB_1000.bin' #'models/tokenizers/morf_zinc_1000.bin'
morf_vocab_pt = './vocabs/morf_vocab/morf_smilesDB_vocab1000.txt' # './vocabs/morf_vocab/morf_zinc250k_vocab1000.txt'
mofressor_tokenizer = SMILES_SPE_Tokenizer(vocab_file=morf_vocab_pt,
                                           tokenizer_path=morf_model_pt,
                                           tokenizer_type='Morfessor')

# choose tokenizer 
tokenizer = Bpe_tokenizer #MacFrag_tokenizer #mofressor_tokenizer #Bpe_tokenizer

In [3]:
print(mofressor_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))
print(MacFrag_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2) , c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))
print(Bpe_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))
print(Spe_tokenizer._tokenize('CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2)c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F'))



['C', 'S(=O)(=O)', 'C', 'C', 'N', 'C', 'c1ccc(', 'o1', ')', 'c2ccc3c(c2)', 'c(', 'ncn', '3', ')N', 'c4ccc(', 'c(', 'c4', ')', 'Cl)', 'O', 'C', 'c5ccc', 'c(', 'c5', ')', 'F']
['CS(=O)(=O)CCNCc1ccc(o1)c2ccc3c(c2) ', ' c(ncn3)Nc4ccc(c(c4)Cl)OCc5cccc(c5)F']
['CS(=O)(=O)', 'CCN', 'Cc1ccc(', 'o1)', 'c2ccc3c(c2)', 'c(', 'ncn', '3)N', 'c4ccc(', 'c(', 'c4)C', 'l)', 'OC', 'c5cccc', '(', 'c5)', 'F']
['CS(=O)(=O)', 'CCN', 'Cc1ccc(', 'o1)', 'c2ccc3c(c2)', 'c(', 'ncn', '3)', 'N', 'c4cc', 'c(', 'c(', 'c4)', 'Cl)', 'OC', 'c5', 'cccc', '(', 'c5', ')', 'F']


In [4]:
# prepare datasets 
import pandas as pd
clin_tox_zinc250 = "./datasets/cleaned_smilesDB.smi"
clin = "./datasets/pre_processed/clintox.smi"
zinc  = "./datasets/zinc_smiles.txt"
# read clin_tox_zinc250 smiles and store in a list
with open(clin_tox_zinc250, "r") as f:
         smilesDB = f.read().strip().split("\n")
    
# clintox_frags = './datasets/mac_fragments/clintox_fragments.smi'
# tasks, (train_df, valid_df, test_df), transformers = load_molnet_dataset("clintox", tasks_wanted=None, split='random')
# tasks, (train_df, valid_df, test_df), transformers = load_molnet_dataset("tox21", tasks_wanted= ['NR-AhR'])
clintox_df = pd.read_csv('./datasets/post_processed/clintox_processed.csv' )
tox21_df = pd.read_csv('./datasets/post_processed/tox21_processed.csv' )
# split imbalanced dataset clintox into train, valid, test
train_df, valid_df = train_test_split(clintox_df, test_size=0.2, random_state=42 , stratify=clintox_df['CT_TOX'], shuffle=True)
print("train shape" , train_df.shape, "valid shape" , valid_df.shape)
# Create a dictionary with the text data
tr_data_dict = {"text": train_df['smiles'].tolist()} 
val_data_dict = {"text": valid_df['smiles'].tolist()}

# Create a dataset using the `Dataset` class from `datasets`
tr_dataset= Dataset.from_dict(tr_data_dict)
val_dataset= Dataset.from_dict(val_data_dict) 

print("train dataset shape" , tr_dataset.shape, "valid dataset shape" , val_dataset.shape)
print("Tokenization and encoding")
tr_tokenized_dataset = tr_dataset.map(lambda examples: tokenizer(examples["text"], truncation=True,padding=True, max_length=128), batched=True)
val_tokenized_dataset = val_dataset.map(lambda examples: tokenizer(examples["text"], truncation=True,padding=True, max_length=128), batched=True)
tr_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'text'])
val_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'text'])
# call add_decoded_tokens to add decoded tokens to the data
tr_tokenized_dataset = tr_tokenized_dataset.map(lambda batch: add_decoded_tokens(batch= batch, tokenizer=tokenizer))
val_tokenized_dataset = val_tokenized_dataset.map(lambda batch: add_decoded_tokens(batch= batch, tokenizer=tokenizer))
print("length of train dataset:", len(tr_tokenized_dataset))
print("length of val dataset:", len(val_tokenized_dataset))


train shape (1180, 4) valid shape (296, 4)
train dataset shape (1180, 1) valid dataset shape (296, 1)
Tokenization and encoding


Map:   0%|          | 0/1180 [00:00<?, ? examples/s]

Map:   0%|          | 0/296 [00:00<?, ? examples/s]

Map:   0%|          | 0/1180 [00:00<?, ? examples/s]

Map:   0%|          | 0/296 [00:00<?, ? examples/s]

length of train dataset: 1180
length of val dataset: 296


In [8]:
tr_tokenized_dataset[3]

{'text': 'C[NH+]1CCCC(C1)CC2c3ccccc3Sc4c2cccc4',
 'input_ids': tensor([ 12,  11,  11, 150, 848,  11, 988, 873, 119,  13,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'decoded_tokens': '[CLS] [UNK] [UNK] 1) CC2 [UNK] c4 c2cccc 4 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [6]:

config = RobertaConfig(
    # vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=6,
    num_hidden_layers=3,
    type_vocab_size=1,
)

model = RobertaForMaskedLM(config=config)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

model_name= "Clintox_Bpe_chemBERTa"

training_args = TrainingArguments(
    output_dir="./models/pre_trained_models/"+model_name,
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_steps=10000,
    # save_total_limit=2,
    logging_steps=5,
    evaluation_strategy = "epoch",
    save_strategy = "no",
    seed=42,
    # load_best_model_at_end=True, # requires save strategy to be "none
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tr_tokenized_dataset,
    eval_dataset=val_tokenized_dataset,
    # prediction_loss_only=True,
)

In [7]:
trainer.train()
trainer.save_model("./models/pre_trained_models/Best_"+ model_name)
trainer.evaluate()

/home/isiddig/anaconda3/envs/toxicity/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ibrahim-siddig-i-e. Use `wandb login --relogin` to force relogin


  0%|          | 0/95 [00:00<?, ?it/s]

{'loss': 7.2075, 'learning_rate': 4.736842105263158e-05, 'epoch': 0.26}
{'loss': 2.9651, 'learning_rate': 4.473684210526316e-05, 'epoch': 0.53}
{'loss': 1.9846, 'learning_rate': 4.210526315789474e-05, 'epoch': 0.79}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.7431315183639526, 'eval_runtime': 42.1898, 'eval_samples_per_second': 7.016, 'eval_steps_per_second': 0.119, 'epoch': 1.0}
{'loss': 1.7281, 'learning_rate': 3.9473684210526316e-05, 'epoch': 1.05}
{'loss': 1.5935, 'learning_rate': 3.6842105263157895e-05, 'epoch': 1.32}
{'loss': 1.6496, 'learning_rate': 3.421052631578947e-05, 'epoch': 1.58}
{'loss': 1.5431, 'learning_rate': 3.157894736842105e-05, 'epoch': 1.84}


...

{'eval_loss': 0.9994926452636719, 'eval_runtime': 15.9572, 'eval_samples_per_second': 18.55, 'eval_steps_per_second': 0.313, 'epoch': 4.0}
{'loss': 0.8633, 'learning_rate': 7.894736842105263e-06, 'epoch': 4.21}
{'loss': 1.1417, 'learning_rate': 5.263157894736842e-06, 'epoch': 4.47}
{'loss': 0.9327, 'learning_rate': 2.631578947368421e-06, 'epoch': 4.74}
{'loss': 1.004, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.1040592193603516, 'eval_runtime': 16.2914, 'eval_samples_per_second': 18.169, 'eval_steps_per_second': 0.307, 'epoch': 5.0}
{'train_runtime': 1775.4085, 'train_samples_per_second': 3.323, 'train_steps_per_second': 0.054, 'train_loss': 1.6926688796595524, 'epoch': 5.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.0414453744888306,
 'eval_runtime': 16.2487,
 'eval_samples_per_second': 18.217,
 'eval_steps_per_second': 0.308,
 'epoch': 5.0}